# Camera Picture Generation

In [ ]:
import time
import cv2

pTime = 0
cTime = 0
cap = cv2.VideoCapture(1)

cap.set(3,200)

cap.set(4,200)
# cap.set(cv2.CAP_PROP_EXPOSURE,-6)
success, img = cap.read()
cv2.imshow("Image", img)
cv2.waitKey(1)
i = 0
# while i <= 50:
while True:
    while i <= 400:
        success, img = cap.read()

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.imshow("Image", img)
        cv2.waitKey(1)
        i+=1
        # cv2.putText(img, str(int(fps)), (50,50),3)

        time.sleep(.1)
        print(f"Photo Number {i}")
        isWritten = cv2.imwrite(f'D:\Coding\ML\_tictactoe\O\{i + 401}.png', img)

# Model Stuff

In [ ]:

from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sn
from tensorflow.keras.optimizers import SGD
# config = tf2.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf2.InteractiveSession(config=config)
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.set_logical_device_configuration(
    physical_devices[0],
    tf.config.LogicalDeviceConfiguration(memory_limit=4096))

  logical_devices = tf.config.list_logical_devices('GPU')
  assert len(logical_devices) == len(physical_devices) + 1

  tf.config.set_logical_device_configuration(
    physical_devices[0],
    tf.config.LogicalDeviceConfiguration(memory_limit=4096))
except:
#   # Invalid device or cannot modify logical devices once initialized.
  pass
opt = SGD(lr=0.001, momentum=0.9)
# adam = tf.keras.optimizers.Adam(
#     learning_rate= 0.02,
#     # epsilon=1e-08,
#     amsgrad=False,
#     name="adam"
# )
def define_model():
    
	model = VGG16(include_top=False, input_shape=(200, 200, 3))

	for layer in model.layers:
		layer.trainable = False

	flattener = Flatten()(model.layers[-1].output)
	layer1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flattener)
	output = Dense(2, activation='softmax')(layer1)

	model = Model(inputs=model.inputs, outputs=output)
	model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
	return model


def run():
	# define model
    model = define_model()
        # create data generator
    print("train_dir")
    training_dir = 'D:\Coding\AIsociety\_tictactoe\content\\training'
    image_size = (200, 200)

    train_datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=.2,
            zoom_range=.2,
            rotation_range = 40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            )
    validation_datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=.2,
            rotation_range = 40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            )

    train_generator = train_datagen.flow_from_directory(
            training_dir,
            target_size = image_size,
            subset="training",
            batch_size=32,
            class_mode='sparse',
            seed=42,shuffle=True)

    validation_generator = validation_datagen.flow_from_directory(
            training_dir,
            target_size=image_size,
            batch_size=32,
            class_mode='sparse',
            subset="validation",
            seed=42)
    print("test_dir")
    test_dir = 'D:\Coding\AIsociety\_tictactoe\content'

    test_datagen = ImageDataGenerator(rescale=1./255,zoom_range=.2,
            rotation_range = 40,
            width_shift_range=0.2,
            height_shift_range=0.2)
    test_generator = test_datagen.flow_from_directory(
            test_dir,
            target_size=(200, 200),
            classes=['test'],
            class_mode='sparse',
            shuffle=False)
        # fit model
    history = model.fit(train_generator, steps_per_epoch=len(train_generator),
        validation_data=validation_generator, validation_steps=len(validation_generator), epochs=14, verbose=1)
    # evaluate model
    plt.plot(history.history['sparse_categorical_accuracy'])
    plt.plot(history.history['val_sparse_categorical_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    base = pd.read_csv("D:\Coding\AIsociety\_tictactoe\content\\test_classes.csv")
    base = base.ClassId


    pred = model.predict(test_generator, verbose=1)

    cl = []
    for i in range(0, len(pred)):
        cl.append(np.argmax(pred[i]))
    cl = np.array(cl)
    print(cl[0])
    # for i in range(0,802):
    #     if cl[i] != base[i]:
    #         print("number")
    #         print(i)
    #         print("cl")
    #         print(cl[i])
    #         print("base")
    #         print(base[i])
    Accuracy = accuracy_score(base, cl)
    
    print("accuracy on Test:", Accuracy)
    from sklearn import metrics
    print("Accuracy:",metrics.accuracy_score(base, cl))
    print("Recall: ", metrics.recall_score(base, cl, average='micro'))

    cm = confusion_matrix(base,cl)
    df_cm = pd.DataFrame(cm, range(2), range(2))
    # plt.figure(figsize=(10,7))
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

    plt.show()
    i = 0
    print(pred[i])
    plt.figure(figsize=(20,20))
    plt.subplot(1,2,1)
    plot_image(i, pred[i], base, gen_file(i))
    plt.subplot(1,2,2)
    plot_value_array(i, pred[i],  base)
    plt.show()

    i = 1
    print(pred[i])
    plt.figure(figsize=(20,20))
    plt.subplot(1,2,1)
    plot_image(i, pred[i], base, gen_file(i))
    plt.subplot(1,2,2)
    plot_value_array(i, pred[i],  base)
    plt.show()

run()

I built this to compare the VGG model

In [ ]:
model = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation=LeakyReLU(alpha=0.05), input_shape=(200,200,3),kernel_initializer = 'he_uniform')),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation=LeakyReLU(alpha=0.05),kernel_initializer = 'he_uniform')),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation=LeakyReLU(alpha=0.05),kernel_initializer = 'he_uniform')),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation=LeakyReLU(alpha=0.05),kernel_initializer = 'he_uniform')),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dropout(0.3),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation=LeakyReLU(alpha=0.05)),
    # tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

## Some Project code that this runs with to see Board_state

This code is used to both see the grid, split the grid, and make sorted individual photos to send to the model

In [ ]:
import cv2
import numpy as np
FILENAME = 'D:\Coding\pick.png'
def get_photo():
    cap = cv2.VideoCapture(1) # video capture source camera (Here webcam of laptop) 
    while(True):
      ret,frame = cap.read()
      cv2.imshow('img1',frame) #display the captured image
      if cv2.waitKey(1) & 0xFF == ord('y'): #save on pressing 'y' 
          cv2.imwrite(FILENAME,frame)
          cv2.destroyAllWindows()
          break

    cap.release()
    return
get_photo()

def blob_distance():
# (hMin = 163 , sMin = 59, vMin = 116), (hMax = 179 , sMax = 234, vMax = 255)
  img = cv2.imread("D:\Coding\pick.png", cv2.IMREAD_COLOR)
  hMin = 163
  sMin = 59
  vMin = 116

  hMax = 179
  sMax = 234
  vMax = 255

  # Set minimum and max HSV values to display
  lower = np.array([hMin, sMin, vMin])
  upper = np.array([hMax, sMax, vMax])
  canvas = img.copy()
  # Create HSV Image and threshold into a range.
  hsv = cv2.cvtColor(canvas, cv2.COLOR_BGR2HSV)
  mask = cv2.inRange(hsv, lower, upper)
  mask = cv2.bitwise_not(mask)
  # blur = cv2.blur(mask, (5,5), 0)
  cv2.imshow("Mask", mask)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
  params = cv2.SimpleBlobDetector_Params()

  params.minThreshold = 10
  params.maxThreshold = 200

  # Filter by Area.
  params.filterByArea = True
  params.minArea = 10

  # Filter by Circularity
  params.filterByCircularity = False
  params.minCircularity = 0.1

  # Filter by Convexity
  params.filterByConvexity = False
  params.minConvexity = 0.87

  # Filter by Inertia
  params.filterByInertia = False
  params.minInertiaRatio = 0.01
  detector = cv2.SimpleBlobDetector_create(params)

  keypoints = detector.detect(mask)
  
  keys = cv2.drawKeypoints(img, keypoints, np.array([]),(0,0,255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
  cv2.imshow("Keys", keys)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
  return keypoints



def distance(kpt1, kpt2):
    #create numpy array with keypoint positions
    arr = np.array([kpt1.pt, kpt2.pt])
    # print(kpt1.pt)
    # print(kpt2.pt)
    #scale array to mm
    #return distance, calculted by pythagoras
    return np.sqrt(np.sum((arr[0]-arr[1])**2))


def area(keypoints):
  for i,keypoint in enumerate(keypoints[1:]):
      print("Distance: {0:6.3f}".format(distance(keypoints[0], keypoint)))
  dist = distance(keypoints[0], keypoint)
  area = .5*(dist**2)
  return area


keypoints = blob_distance()
print(keypoints)
area(keypoints)


def grid(keypoints):
  
  image = FILENAME
  image = cv2.imread(image,cv2.IMREAD_COLOR)
  for i in enumerate(keypoints[1:]):
    dot_1 = keypoints[1].pt
    dot_2 = keypoints[0].pt
  print("dot_1")
  print(dot_1)
  print("dot_2")
  print(dot_2)
  lx = min[dot_1[0], dot_2[0]]
  ly = min[dot_1[1], dot_2[1]]
  lw = max[dot_1[0], dot_2[0]]
  lh = max[dot_1[1], dot_2[1]]

  lx = int(lx)
  ly = int(ly)

  lw = int(lw)
  lh = int(lh)
  # lx = int(dot_1[0])
  # ly = int(dot_1[1])

  # lw = int(dot_2[0])
  # lh = int(dot_2[1])
  print('lx,ly,lw,lh')
  print(lx,ly,lw,lh)


  print('lx,ly,lw,lh')
  print(lx,ly,lw,lh)
  rect = cv2.rectangle(image, (lx,ly), (lw,lh), (255, 255, 0), 5, cv2.LINE_AA)
  print(rect)
  largest = image[ly:lh, lw:lx]
  print(largest.shape)
  
  
  # ret, img = image.read()
  # cv2.imshow('img', img)
  # k = cv2.waitKey(30) & 0xff
  # if k == 27:
  #     break
  # ret, image = img.read()

  gray = cv2.cvtColor(largest, cv2.COLOR_BGR2GRAY)
  # cv2.imshow("gray", gray)

  blur = cv2.GaussianBlur(gray, (5,5), 0)
  # cv2.imshow("blur", blur)

  thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)
  # cv2.imshow("thresh", thresh)

  contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


  big = max(contours, key = cv2.contourArea)
  print('big.shape')
  print(big.shape)
  largest_area = cv2.contourArea(big)
  print(largest_area)

  bx,by,bw,bh = cv2.boundingRect(big)
  print("bx,by,bw,bh")
  print(bx,by,bw,bh)
  biggest = largest[by:by+bh, bx:bx+bw]
  print('biggest.shape')
  print(biggest.shape)
  print('[by,by+bh]')
  print([by,bh])
  print('bx,bx+bw')
  print(bx,bw)
  cv2.rectangle(largest, (bx,by), (bx+bw, by+bh), (0, 255, 0), 2, cv2.LINE_AA)
  c = 0
  n = 0
  for i in contours:
          # x,y,w,h = cv2.boundingRect(i)
          # if x < bx and (((w)*3)/(bw) < 1) and (((w)*3)/(bw) > .8):
          area = cv2.contourArea(i)
          if area >= (largest_area / 13) and area <= (largest_area / 6):
            x,y,w,h = cv2.boundingRect(i)
            cv2.putText(largest, f'{8-n}', (int(x+(w/2-5)),int(y+(h/2))), cv2.FONT_HERSHEY_DUPLEX, .4, color=(255, 0, 0) )
            cropped = largest[y:y+h, x:x+w]

            cv2.imshow(f"Crop{8-n}", cropped)
            cv2.imwrite(f'D:\Coding\AIsociety\_tictactoe\Matrix-{8-n}.png', cropped)

            cv2.rectangle(largest, (x,y), (x+w, y+h), (0, 255, 0), 2, cv2.LINE_AA)
            n+=1   
            c+=1
  # cv2.drawContours(img, contours, -1, (0, 255, 0), 3)
  cv2.imshow("Image", largest)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #       break
grid(keypoints)